In [ ]:
import zipfile
names = []
with zipfile.ZipFile("pgns.zip", "r") as f:
    for name in f.namelist():
        names.append(name)

In [ ]:
names = [s.replace('__MACOSX/', '') for s in names]

In [ ]:
import datefinder
from tabula import read_pdf
import locale
import tabula

locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 
wrongformats = []
names_errors = []
list1 = []


for x in names:
    try:
        # 2008-2009 style
        tables = tabula.read_pdf(x, pages = 2, multiple_tables = True)
        df = tables[-1]
        df['try1'] = df[df.columns[1]].str.split()
        df = df.set_index(df.columns[0]).T 
        
        total_auctions_planned = locale.atoi(df['Total auctions planned'][-1][-1])
        total_auctions_remaining = locale.atoi(df['Auctions remaining'][-1][-1])
        
        total_planned_sales = locale.atoi(df['Total planned sales'][-1][-1])
        total_sales_remaining = locale.atoi(df['Sales remaining'][-1][-1])
        total_sales_todate = df['Gilt sales to date'][-1][-1]
        
        avg_auction_value = total_planned_sales/total_auctions_planned
        supposed_auction_value_todate = (total_planned_sales/total_auctions_planned) * (total_auctions_planned - total_auctions_remaining)
        
        auction_value_remaining = total_sales_remaining/total_auctions_remaining
                
        matches = datefinder.find_dates(str(tables[-1].columns))
        for match in matches:
            date = match
        
        list1.append([date, avg_auction_value, supposed_auction_value_todate, total_sales_todate,
                     total_auctions_planned,total_auctions_remaining,total_planned_sales,total_sales_remaining])
    
    except:
        try:
            # 2014-2019 styles
            tables = tabula.read_pdf(x, pages = 2, multiple_tables = True)
            df = tables[-1]
            df = df.dropna(subset = [str(df.columns[0])])

            total_auctions_planned = int(df[df[str(df.columns[0])].str.contains("Number of auctions scheduled")][str(df.columns[-1])].values)
            total_auctions_remaining = int(df[df[str(df.columns[0])].str.contains("Number of auctions remaining")][df.columns[-1]].str.split('\r').iloc[0][-1])

            total_planned_sales = locale.atoi(df[df[str(df.columns[0])].str.contains("Total planned gilt sales")][str(df.columns[-1])].iloc[0])
            total_sales_remaining = total_planned_sales - locale.atoi(df[df[str(df.columns[0])].str.contains("Total gilt sales to date")][str(df.columns[-1])].iloc[0])
            total_sales_todate = locale.atoi(df[df[str(df.columns[0])].str.contains("Total gilt sales to date")][str(df.columns[-1])].iloc[0])

            avg_auction_value = total_planned_sales/total_auctions_planned
            supposed_auction_value_todate = (total_planned_sales/total_auctions_planned) * (total_auctions_planned - total_auctions_remaining)

            auction_value_remaining = total_sales_remaining/total_auctions_remaining

            matches = datefinder.find_dates(str(df.columns[0]))
            for match in matches:
                date = match

            list1.append([date, 
                          avg_auction_value, 
                          supposed_auction_value_todate, 
                          total_sales_todate,
                         total_auctions_planned,
                          total_auctions_remaining,
                          total_planned_sales,
                          total_sales_remaining,
                         auction_value_remaining])

        except:
            try:
                # 2020-2021 styles remits
                tables = tabula.read_pdf(x, pages = 2, multiple_tables = True)
                df = tables[-1]
                df = df.dropna(subset = [str(df.columns[0])])

                total_planned_sales = locale.atoi(df[df[df.columns[0]].str.contains("Planned gilt sales")][df.columns[1]].iloc[0])
                total_sales_todate = locale.atoi(df[df[df.columns[0]].str.contains("Total gilt sales to date")][str(df.columns[1])].iloc[0])
                total_sales_remaining = total_planned_sales - total_sales_todate

                total_auctions_planned = 12*2
                total_auctions_remaining = 3*2

                avg_auction_value = total_planned_sales/total_auctions_planned
                supposed_auction_value_todate = (total_planned_sales/total_auctions_planned) * (total_auctions_planned - total_auctions_remaining)

                auction_value_remaining = total_sales_remaining/total_auctions_remaining

                matches = datefinder.find_dates(str(df.columns[0]))

                for match in matches:
                    date = match

                list1.append([date, avg_auction_value, 
                              supposed_auction_value_todate, 
                              total_sales_todate,
                             total_auctions_planned,
                              total_auctions_remaining,
                              total_planned_sales,
                              total_sales_remaining,
                             auction_value_remaining])

            except:
                wrongformats.append(x)
                pass

In [ ]:
import pickle
with open("list1finally.txt", "wb") as fp:   #Pickling
    pickle.dump(list1, fp)